In [1]:
# NLP Lecture @ Strive School - 21st July 2021
# NER update

'''
Since today we are exploring the world of natural language processing, we’ll deepen in the Named Entity Recognition technique:
this is just one of the mechanisms that NLP embodies. The recognition of named entities as the process of automatic
identification of the entities present in a text and consequent classification into predefined categories such as "person",
"organization", "position" is a quite common activity and expect for English, trained models with spaCy offer few labels that
could be improved through training.

Following the case study of this morning, try to emulate it in order to label all the brands present in the provided datasets,
choosing the one you prefer OR trying to label all them and to train the model to recognize new different entities.
The result should be twofold: the final model should be able to recognize brands that it has already seen, but already new ones.
The brands proposed in the dataset concern fashion, cars and food.
In order to test the accuracy of the model, test it with sentences and brands the model has never seen.

Sample of the dataset
---------------------
- Cate Blanchett in Armani Privé. Rating: 8. Concludes as a rare butterfly, or from Rorschach's Test, or from computerized
axial tomography.
- I liked everything, recommend it! Another quality Xiaomi product...
- What is the price of that Fiat 500XL?

Info:
- Feel free to change or arrange a new dataset
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)

'''

'\nSince today we are exploring the world of natural language processing, we’ll deepen in the Named Entity Recognition technique:\nthis is just one of the mechanisms that NLP embodies. The recognition of named entities as the process of automatic\nidentification of the entities present in a text and consequent classification into predefined categories such as "person",\n"organization", "position" is a quite common activity and expect for English, trained models with spaCy offer few labels that\ncould be improved through training.\n\nFollowing the case study of this morning, try to emulate it in order to label all the brands present in the provided datasets,\nchoosing the one you prefer OR trying to label all them and to train the model to recognize new different entities.\nThe result should be twofold: the final model should be able to recognize brands that it has already seen, but already new ones.\nThe brands proposed in the dataset concern fashion, cars and food.\nIn order to test t

In [2]:
# STEP 0 - PRE REQUISITES

# python -m spacy download en_core_web_lg

# TBD: Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TBD: Load preferred model

nlp = spacy.load('en_core_web_lg')

# TBD: Load the dataset and test it as-is

with open("electronics.txt") as file:
    dataset = file.read()

print(dataset)

doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])



I am being forced to purchase a new device as all US networks are apparently no longer supporting Huawei devices in the very near future.
Samsung S20 is The best Phone that money can buy
I'm a Samsung fan, design and productivity is amazing and perfect, BUT it have a no-go for me: it poses a risk for phone owners.
The hardware is absolutetly stellar, which is no surprise for Samsung in the past years.
Unfortunately, same goes for app bloatware, which is also no surprise for Samsung.
 The phone feels amazing to the touch. Haven't messed with the camera much but from what I have I can say it's not half bad. I bought a micro sdxc card from Samsung to put all my movies and music on and use VLC to watch/listen and it's great.
Last time I checked samsung had there high end devices free to unlock there bootloader. At least from the s7 days .
Techniques used in Huawei is great, and even awesome.
huawei really did an amazing job.
Huawei phones are excellent. I have used them for ever, unfortuna

In [3]:
# STEP 1 - TRAIN DATA

# Prepare training data

# TBD: define all the entities by extracting the words and their indexes from the dataset
# expected format is the following:  ("sentence", {"entities": [0,10, "FOOD"]})

words = ['xiaomi', 'samsung', 'nokia', 'apple', 'huawei', 'at&t', 'redmi', 'mediatk', 'oppo', 'motorola', 'oneplus', 'google pixel',
        'tcl', 'lg'
]

train_data = []

with open("electronics.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        print("######")
        print("sentence: ", sentence)
        print("######")
        sentence = sentence.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sentence:
                start_index = sentence.index(word)
                end_index = len(word) + start_index
                print("word: ", word)
                print("----------------")
                print("start index:", start_index)
                print("end index:", end_index)
                pos = (start_index, end_index, "ORG")
                entities.append(pos)
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
        print('----------------')
        print("element:", element)

# STEP 2 - UPDATE MODEL

# TBD: load the needed pipeline
ner = nlp.get_pipe("ner")

# TBD: define the annotations
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# TBD: train the model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# TBD: define the number of iterations, the batch size and the drop according to your experience or using an empirical value
# Train model

iter = 500
drop_out = 0.4

with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(iter):
        print("Iteration #" + str(iteration))

        # Data shuffle for each iteration
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                # Create an Example object
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=drop_out)
        print("Losses:", losses)

# Save the model
# TBD:

output_dir = Path("./ner/")
nlp.to_disk(output_dir)
print("Saved correctly!")

######
sentence:  I am being forced to purchase a new device as all US networks are apparently no longer supporting Huawei devices in the very near future.

######
word:  huawei
----------------
start index: 98
end index: 104
----------------
element: ('i am being forced to purchase a new device as all us networks are apparently no longer supporting huawei devices in the very near future.', {'entities': [(98, 104, 'ORG')]})
######
sentence:  Samsung S20 is The best Phone that money can buy

######
word:  samsung
----------------
start index: 0
end index: 7
----------------
element: ('samsung s20 is the best phone that money can buy', {'entities': [(0, 7, 'ORG')]})
######
sentence:  I'm a Samsung fan, design and productivity is amazing and perfect, BUT it have a no-go for me: it poses a risk for phone owners.

######
word:  samsung
----------------
start index: 6
end index: 13
----------------
element: ("i'm a samsung fan, design and productivity is amazing and perfect, but it have a no

In [6]:
# STEP 3 - TEST THE UPDATED MODEL

# Load updated model
print("Loading model...")
nlp_updated = spacy.load(output_dir)

# TBD: test with a old sentence
doc = nlp_updated('After a couple of months from my purchase I tell you a bit of Xiaomi mi10t.')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# TBD: test with a new sentence and an old brand
doc = nlp_updated('I wll buy the new Apple thing')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# TBD: test with a new sentence and a new brand
doc = nlp_updated('I find the new Doogee really good')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

Loading model...
entities: [('Xiaomi', 'ORG')]
entities: [('Apple', 'ORG')]
entities: [('Doogee', 'ORG')]
